The master_file is an example of how the address_compare library can be used to load training and test data, tag the addresses, standardize the addresses, and compare the different address lists.  It can serve as a reusable program by updating the input parameters.  If ground truth files are available, it will also show how well the tagger and compare functions perform.

In [5]:
#from collections import defaultdict
#from collections import OrderedDict
#from address_compare import standardizers as stndrdzr
#from address_compare import matcher as mtch
#from address_compare.crf_tagger import AddressTagger
#from address_compare import address_randomizer as add_rndm
from address_compare import aggregate_functions as aggf
import json
import pandas as pd

Although this notebook is an example of how to use the address_compare library, the following parameters can be changed in order to control the inputs and outputs.  I.e., the following parameters allow this file to be a reusable program sitting on top of the address_compare library.


The **run_mode** variable controls which portions of this notebook are run.  Options are:
- **'tagger'** = run the address tagger against a single file that also contains the ground truths.  output will show how well the tagger did against the ground truths.  tagger will only run against the file found in @file_location_raw_addresses_1
- **'comparer'** = tag 2 separate lists of addresses and find matches between the lists.  no ground truths for comparisons.  no tagger ground truths or match ground truths included. program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2
- **'comparer_truths'** = run the comparer and validate the matcher performance against the ground truths. program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2.  in addition, the matched ground truths will be found in @file_name_ground_truth_matches
- **'all'** = runs all 3 modes.  i.e., tagger results compared against the ground truths and the matcher results compared against the ground truths.  program will run against both @file_location_raw_addresses_1 and @file_location_raw_addresses_2. in addition, the matched ground truths will be found in @file_name_ground_truth_matches

In [6]:
retrain_crf_tagger = False #if True, the specified training file will be used to retrain the CRF Tagger

run_mode = 'tagger' #choose from ['tagger','comparer','comparer_truths','all']


standardize_addresses = True #if True, the tagged address components will be standardized (changed to upper case, unit types, street types, etc. changed to long form names)

use_raw_address_files = True #if False, only the specified number of randomly created addresses above will be used
num_rndm_addresses_to_create = 100 #if use_raw_address_files = False, the number of addresses that will be randomly created for use in the tagger and compare functions

field_name_raw_addresses = 'Single String Address' #represents the name of the field in the raw address files containing the raw address (street information)
field_name_record_id = 'Record_ID' #represents the name of the field containing the Record ID in the raw files; if not present in the raw files, populate with None

#file_location_raw_addresses_1 = 'data\\standardized tagged washington state addresses.xlsx'
file_location_raw_addresses_1 = 'data\\tagged standardized colorado Stores.xlsx'
#file_location_raw_addresses_1 = 'data\\MarijuanaApplicants - test data list 1.xlsx'
file_location_raw_addresses_2 = 'data\\MarijuanaApplicants - test data list 2.xlsx'

file_name_ground_truth_matches = 'data\\marijuana applicants test data - correct matches.xlsx'

write_output_to_excel = True #if True, the output from the applicable modes will be written to Excel

In [7]:
# Placeholder for reading/calling the training data for the CRF Tagger and sending the training data to train the model
if retrain_crf_tagger:
    with open('data/tagged_addresses.json') as f:
        td = json.load(f)
    
    #send training data to CRF tagger to train the model here...

In [8]:
if run_mode == 'tagger':
    df_dict = aggf.tagger_vs_ground_truths(file_location_raw_addresses_1, field_name_record_id, field_name_raw_addresses, standardize_addresses)
    if write_output_to_excel:
        output_name = 'output\\file_1_tagger_vs_truths.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in df_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
    else:
        for sheet, frame in df_dict.items():
            print ("sheet name = ", sheet)
            print (frame)

sheet name =  test_data_file
    Record_ID                 Single String Address           CITY STATE  \
0           1                      17034 Highway 17         MOFFAT    CO   
1           2               13621 Washington Street       THORNTON    CO   
2           3             1617 Wazee Street, Unit B         DENVER    CO   
3           4           155 North Federal Boulevard         DENVER    CO   
4           5                 4012 West 38th Avenue         DENVER    CO   
5           6                248 East US Highway 50         SALIDA    CO   
6           7         6403 North Jackson Gap Street         AURORA    CO   
7           8                    1401 Peoria Street         AURORA    CO   
8           9                   106 East 4th Avenue       ANTONITO    CO   
9          10               4625 East Colfax Avenue         DENVER    CO   
10         11                  505 West 40th Avenue         DENVER    CO   
11         12                      5110 Race Street        

SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
if run_mode in ['comparer','comparer_truths']:
    compared_dict, matcher_truths_dict = aggf.tag_and_compare_addresses(file_location_raw_addresses_1, file_location_raw_addresses_2, file_name_ground_truth_matches, use_raw_address_files, num_rndm_addresses_to_create, field_name_record_id, field_name_raw_addresses, standardize_addresses, run_mode)
    if write_output_to_excel:
        output_name = 'output\\raw_to_matched_addresses.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in compared_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
    else:
        for sheet, frame in compared_dict.items():
            print ("sheet name = ", sheet)
            print (frame)

    if run_mode == 'comparer_truths':
        if write_output_to_excel:
            output_name = 'output\\modeled_matches_vs_ground_truths.xlsx'
            tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
            for sheet, frame in matcher_truths_dict.items():
                frame.to_excel(tagger_writer, sheet_name=sheet)
            tagger_writer.save()
        else:
            for sheet, frame in matcher_truths_dict.items():
                print ("sheet name = ", sheet)
                print (frame)

In [ ]:
if run_mode == 'all':
    tag_truths1_dict, tag_truths2_dict, compared_dict, matcher_truths_dict = aggf.tag_vs_truths_and_compare_addresses(file_location_raw_addresses_1, file_location_raw_addresses_2, file_name_ground_truth_matches, use_raw_address_files, num_rndm_addresses_to_create, field_name_record_id, field_name_raw_addresses, standardize_addresses, run_mode)
    if write_output_to_excel:
        output_name = 'output\\file_1_tagger_vs_truths.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in tag_truths1_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
        
        output_name = 'output\\file_2_tagger_vs_truths.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in tag_truths2_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
        
        output_name = 'output\\raw_to_matched_addresses.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in compared_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
        
        output_name = 'output\\modeled_matches_vs_ground_truths.xlsx'
        tagger_writer = pd.ExcelWriter(output_name, engine='xlsxwriter')
        for sheet, frame in matcher_truths_dict.items():
            frame.to_excel(tagger_writer, sheet_name=sheet)
        tagger_writer.save()
    else:
        for sheet, frame in tag_truths1_dict.items():
            print ("sheet name = ", sheet)
            print (frame)

        for sheet, frame in tag_truths2_dict.items():
            print ("sheet name = ", sheet)
            print (frame)
            
        for sheet, frame in compared_dict.items():
            print ("sheet name = ", sheet)
            print (frame)
            
        for sheet, frame in matcher_truths_dict.items():
            print ("sheet name = ", sheet)
            print (frame)